In [1]:
import pandas as pd
import numpy as np
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import StandardScaler

In [2]:
data = pd.read_csv("../Train_knight.csv")

In [3]:
print(data)

     Sensitivity  Hability  Strength   Power  Agility  Dexterity  Awareness  \
0         11.800     17.26     75.26   431.9  0.09087    0.06232   0.028530   
1         19.810     22.15    130.00  1260.0  0.09831    0.10270   0.147900   
2         10.970     17.20     71.73   371.5  0.08915    0.11130   0.094570   
3         11.620     18.18     76.38   408.8  0.11750    0.14830   0.102000   
4         11.260     19.83     71.30   388.1  0.08511    0.04413   0.005067   
..           ...       ...       ...     ...      ...        ...        ...   
393        8.618     11.79     54.34   224.5  0.09752    0.05272   0.020610   
394       13.530     10.94     87.91   559.2  0.12910    0.10470   0.068770   
395       11.640     18.33     75.17   412.5  0.11420    0.10170   0.070700   
396        9.876     19.40     63.95   298.3  0.10050    0.09697   0.061540   
397       19.550     28.77    133.60  1207.0  0.09260    0.20630   0.178400   

     Prescience  Reactivity  Midi-chlorien  ...  Ev

In [16]:
numeric_cols = data.select_dtypes(include=[np.number]).columns.tolist()
X = data[numeric_cols]

# Standarize data
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

# Calculate VIF (variance inflaction factor)
def calculate_vif(data):
    vif_data = pd.DataFrame()
    vif_data["feature"] = data.columns
    vif_data["VIF"] = [variance_inflation_factor(data.values, i) for i in range(len(data.columns))]
    vif_data["tolerance"] = 1 / vif_data["VIF"]
    return vif_data.sort_values('VIF', ascending=False)

# Show initial VIF
print("initial VIF (standarized data):\n")
print(calculate_vif(X_scaled).to_string(index=False))

# Function for removing VIF above thresold
def remove_high_vif(data, threshold=5):
    features = data.columns.tolist()
    while True:
        vif = calculate_vif(data[features])
        if vif['VIF'].max() > threshold:
            exclude = vif.loc[vif['VIF'].idxmax(), 'feature']
            features.remove(exclude)
        else:
            break
    return features

selected_features = remove_high_vif(X_scaled)

final_vif = calculate_vif(X_scaled[selected_features])
print("\n\nSelected features and final VIF:\n")
print(final_vif.to_string(index=False))

print("\n\nSelected features:\n")
print(selected_features)

initial VIF (standarized data):

      feature         VIF  tolerance
     Strength 4489.653450   0.000223
  Sensitivity 4367.233204   0.000229
     Recovery  782.388684   0.001278
        Power  411.965245   0.002427
       Sprint  359.129874   0.002785
        Stims  346.022133   0.002890
        Slash   80.754427   0.012383
         Pull   73.911907   0.013530
    Awareness   69.852627   0.014316
   Prescience   54.712644   0.018277
    Dexterity   54.126657   0.018475
   Lightsaber   47.550260   0.021030
    Empowered   36.224859   0.027605
        Delay   35.987392   0.027788
   Attunement   33.904833   0.029494
     Grasping   19.758797   0.050610
   Friendship   19.040267   0.052520
        Evade   17.630598   0.056720
      Repulse   15.259597   0.065533
Midi-chlorien   14.650093   0.068259
     Blocking   14.581072   0.068582
     Hability   11.188825   0.089375
         Mass   10.477189   0.095445
        Combo    9.860934   0.101410
        Burst    8.649491   0.115614
     